In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd ./gdrive/MyDrive/DOD\ Dataset

/content/gdrive/MyDrive/DOD Dataset


In [ ]:
!pip install transformers
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=b1c1a6fdf2a6c246099ef20b3dc322765d3dd97c1dcd5e0a4461b7d8a7afc50b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from torch.utils.data import Dataset, random_split
from pathlib import Path
from pandas import read_csv, concat
import numpy as np



class ScopeDataset(Dataset):
    def __init__(self, data_dir: str):
        data_dir = Path(data_dir)

        type_df = read_csv(data_dir / "entailment_scope_negative_sampling.csv")
        self.len = len(type_df)
        self.examples = type_df


    def __getitem__(self, i):
        label = self.examples.iloc[i].label

        return {
            "p1": self.examples.iloc[i].p1,
            "p2": self.examples.iloc[i].p2,
            "label": label,
        }

    def __len__(self):
        return len(self.examples)

In [ ]:
dataset = ScopeDataset('./')

In [ ]:
positives = [x for x in dataset if x['label'] == 1]
negatives = [x for x in dataset if x['label'] == 0]

In [ ]:
len(dataset), len(positives), len(negatives)

(13232, 6616, 6616)

In [ ]:
import random
def getNegativeExamples(data):
    count = len(data)
    neg = []

    while count > 0:
      element1 = random.choice(data)
      p11 = element1['p1']
      
      element2 = random.choice(data)
      p12 = element2['p1']
      p22 = element2['p2']

      if p11 != p12 and p11 != p22:
        new_e = {}
        new_e['p1'] = p11
        new_e['p2'] = element2['p2']
        new_e['label'] = 0

        neg.append(new_e)

        count -= 1

    return neg

In [ ]:
positives = [x for x in dataset if x['label'] == 1]
negatives = [x for x in dataset if x['label'] == 0]

NameError: ignored

In [ ]:
len(dataset)

16708

In [ ]:
positives = ScopeDataset('./')

In [ ]:
len(positives)

6616

In [ ]:
positives[5000]

{'p1': 'Applies to: (1) OSD, the Military Departments, the Office of the Chairman of the Joint Chiefs of Staff and the Joint Staff, the Combatant Commands, the Office of the Inspector General of the Department of Defense, the Defense Agencies, the DoD Field Activities, and all other organizational entities within the DoD (referred to collectively in this issuance as the “DoD Components”). (2) WLAN devices, systems, and technologies developed by commercial industry in compliance with the current Institute of Electrical and Electronics Engineers (IEEE) standard in IEEE Standard 802.11-2016 and ratified amendments and revisions, that are used to store, process, receive, or transmit unclassified and classified information, which will be referred to as “IEEE 802.11.” This also includes the International Organization for Standardization (ISO)/International Electrotechnical Commission 8802-11: 2012 and ratified amendments and revisions for the international operational environment. (3) WLAN-e

In [ ]:
negatives = getNegativeExamples(positives)
len(negatives)

6616

In [ ]:
len(negatives)

6616

In [ ]:
dataset = positives + negatives

In [ ]:
len(dataset)

13232

In [ ]:
data = [x for x in dataset]

In [ ]:
len(data)

13232

In [ ]:
import pandas as pd

df = pd.DataFrame(data)


In [ ]:
df.to_csv('entailment_scope_negative_sampling.csv')

In [ ]:
train_dataset = []
val_dataset = []
test_dataset = []

train_num = 60*len(positives) / 100 
val_num = 15*len(positives) / 100 

for i in range(len(positives)):
    if i <= train_num:
        train_dataset.append(positives[i])
        train_dataset.append(negatives[i])
    
    elif (len(positives) - i) <= val_num:
        val_dataset.append(positives[i])
        val_dataset.append(negatives[i])
    
    else:
        test_dataset.append(positives[i])
        test_dataset.append(negatives[i])

    


In [ ]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

7940 1984 3308


In [ ]:
pos = [x for x in test_dataset if x['label'] == 1]
neg = [x for x in test_dataset if x['label'] == 0]

print(len(pos), len(neg))

1654 1654


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

# mydataset = Dataset.from_generator(dataset) 

myTraindataset = Dataset.from_list(train_dataset)
myValdataset = Dataset.from_list(val_dataset)
myTestdataset = Dataset.from_list(test_dataset)


mydataset = DatasetDict({"train": myTraindataset, "validation": myValdataset, "test":myTestdataset})

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

#raw_datasets = load_dataset("glue", "mrpc")

# facebook/bart-base
#"bert-base-uncased"
#roberta-base
# nlpaueb/legal-bert-base-uncased
# mukund/privbert

checkpoint = "mukund/privbert"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["p1"], example["p2"], truncation=True)


tokenized_datasets = mydataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/7940 [00:00<?, ? examples/s]

Map:   0%|          | 0/1984 [00:00<?, ? examples/s]

Map:   0%|          | 0/3308 [00:00<?, ? examples/s]

In [ ]:
mydataset

DatasetDict({
    train: Dataset({
        features: ['p1', 'p2', 'label'],
        num_rows: 7940
    })
    validation: Dataset({
        features: ['p1', 'p2', 'label'],
        num_rows: 1984
    })
    test: Dataset({
        features: ['p1', 'p2', 'label'],
        num_rows: 3308
    })
})

In [ ]:
mydataset['train'][0]

{'p1': 'This order directs structural reforms to ensure responsible sharing and safeguarding of classified information on computer networks that shall be consistent with appropriate protections for privacy and civil liberties. Agencies bear the primary responsibility for meeting these twin goals. These structural reforms will ensure coordinated interagency development and reliable implementation of policies and minimum standards regarding information security, personnel security, and systems security; address both internal and external security threats and vulnerabilities; and provide policies and minimum standards for sharing classified information both within and outside the Federal Government. These policies and minimum standards will address all agencies that operate or access classified computer networks, all users of classified computer networks (including contractors and others who operate or access classified computer networks controlled by the Federal Government), and all clas

In [ ]:
from transformers import TrainingArguments, Trainer, logging

# training_args = TrainingArguments("test-trainer")

default_args = {
    "output_dir": "./",
    "evaluation_strategy": "epoch",
    "num_train_epochs": 6,
    # "log_level": "error",
    "logging_steps": 1,
    "log_level" : "info",
    "report_to": "none",
}

training_args = TrainingArguments(per_device_train_batch_size=64,
                                  # gradient_accumulation_steps=32,
                                  gradient_checkpointing=True,
                                  # optim="adafactor",
                                  **default_args,
                                )

In [ ]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=True,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
igno

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at mukund/privbert were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at mukund/privbert and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_metric
from sklearn.metrics import mean_squared_error
import evaluate
import numpy as np


accuracy = evaluate.load("accuracy")

# imdb = load_dataset("imdb")

f1 = evaluate.load("f1")

# imdb = load_dataset("imdb")


#F1
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    microF1 = f1.compute(predictions=predictions, references=labels, average='micro')
    macroF1 = f1.compute(predictions=predictions, references=labels, average='macro')
    weightF1 = f1.compute(predictions=predictions, references=labels, average='weighted')

    return {"micro-F1": microF1, "macroF1": macroF1, "weightF1": weightF1}


# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer, TrainerCallback
import torch


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    # callbacks=[GPUCallback()]
)

In [ ]:
!nvidia-smi

Sat May  6 14:44:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    28W /  70W |   1397MiB / 15360MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: p1, p2. If p1, p2 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7,940
  Num Epochs = 6
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 750
  Number of trainable parameters = 124,647,170
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster th

Epoch,Training Loss,Validation Loss,Micro-f1,Macrof1,Weightf1
1,0.156500,2.841193,{'f1': 0.44506048387096775},{'f1': 0.30798744332054406},{'f1': 0.30798744332054406}
2,0.010000,3.241308,{'f1': 0.4647177419354839},{'f1': 0.31727460426703374},{'f1': 0.31727460426703374}
3,0.030800,3.823263,{'f1': 0.48185483870967744},{'f1': 0.3251700680272109},{'f1': 0.3251700680272109}
4,0.001500,4.109864,{'f1': 0.47983870967741943},{'f1': 0.3242506811989101},{'f1': 0.3242506811989101}
5,0.000700,4.498994,{'f1': 0.4889112903225806},{'f1': 0.32836831415030465},{'f1': 0.32836831415030465}
6,0.001000,4.662051,{'f1': 0.48336693548387094},{'f1': 0.32585796805980294},{'f1': 0.32585796805980294}


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: p1, p2. If p1, p2 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1984
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: p1, p2. If p1, p2 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1984
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: p1, p2. If p1, p2 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1984
  B

TrainOutput(global_step=750, training_loss=0.14586380907423638, metrics={'train_runtime': 6331.1434, 'train_samples_per_second': 7.525, 'train_steps_per_second': 0.118, 'total_flos': 1.25337678997512e+16, 'train_loss': 0.14586380907423638, 'epoch': 6.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to ./
Configuration saved in ./config.json
Model weights saved in ./pytorch_model.bin
tokenizer config file saved in ./tokenizer_config.json
Special tokens file saved in ./special_tokens_map.json


In [ ]:
from transformers import Trainer, TrainerCallback
import torch


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    # callbacks=[GPUCallback()]
)

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: p1, p2. If p1, p2 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3308
  Batch size = 8


{'eval_loss': 4.4552903175354,
 'eval_micro-F1': {'f1': 0.4918379685610641},
 'eval_macroF1': {'f1': 0.34722393252893813},
 'eval_weightF1': {'f1': 0.3472239325289382},
 'eval_runtime': 94.5484,
 'eval_samples_per_second': 34.987,
 'eval_steps_per_second': 4.379}

In [ ]:
!pip install captum
!pip install transformers-interpret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 KB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.6 MB/s eta 0:00:00


In [ ]:
train_dataset[1551]

{'text': "\n777 Relax Spa\n4310 w lllinois Ave #240. Midland. TX 79703\nOpen 7 days 9:00am-10:00pm\nTel: 432-934-2797\nNew and beautiful ladies waiting to pamper you with the best skills available anywhere.\nWe are highly trained in the art of pleasure and we know how to please even the most demanding man.\nWe will rub our delicious bodies all over you as we massage you with the pleasure that will blow your mind.\nWe know what you're looking for and we are here to give it to you.\nWe are the beauties you've been searching for.\nWe're not clock watchers like the other places.\nYou won't be disappointed when you see us,that's a promise.\nYou'll be even happier with the wonderful service we offer.\nstop by today and give us a try .\n\xa0\n\xa0\n\n\n\n\nWe are an agency and serve many cities. Photos are a likeness of the girls presented but may not be 100% accurate.",
 'label': 0}

In [ ]:
text= train_dataset[1551]['text']
text

"\n777 Relax Spa\n4310 w lllinois Ave #240. Midland. TX 79703\nOpen 7 days 9:00am-10:00pm\nTel: 432-934-2797\nNew and beautiful ladies waiting to pamper you with the best skills available anywhere.\nWe are highly trained in the art of pleasure and we know how to please even the most demanding man.\nWe will rub our delicious bodies all over you as we massage you with the pleasure that will blow your mind.\nWe know what you're looking for and we are here to give it to you.\nWe are the beauties you've been searching for.\nWe're not clock watchers like the other places.\nYou won't be disappointed when you see us,that's a promise.\nYou'll be even happier with the wonderful service we offer.\nstop by today and give us a try .\n\xa0\n\xa0\n\n\n\n\nWe are an agency and serve many cities. Photos are a likeness of the girls presented but may not be 100% accurate."

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# With both the model and tokenizer initialized we are now able to get explanations on an example text.

from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)
word_attributions = cls_explainer(text)

In [ ]:
cls_explainer.visualize("bert_vizNeg3.html")

In [ ]:
word_attributions

[('[CLS]', 0.0),
 ('77', -0.009758368134855139),
 ('##7', -0.07936665027486223),
 ('relax', 0.2990892238939921),
 ('spa', -0.12879330936212363),
 ('43', 0.12640561270436662),
 ('##10', -0.04830861860515412),
 ('w', 0.10651616208620975),
 ('ll', 0.27079037332212397),
 ('##lino', 0.39855714189588626),
 ('##is', 0.09783411423537268),
 ('ave', -0.03743073757948999),
 ('#', -0.09033844534034062),
 ('240', 0.026256813339543095),
 ('.', 0.016434362843513267),
 ('midland', -0.1690419568272774),
 ('.', 0.009402800759931669),
 ('tx', -0.3213072447533129),
 ('79', 0.06623658652743815),
 ('##70', -0.18208155651448163),
 ('##3', -0.17815650352317694),
 ('open', -0.030802192742160756),
 ('7', -0.048803319171009184),
 ('days', -0.02326182053718539),
 ('9', 0.005036160876784132),
 (':', 0.02008533454543746),
 ('00', 0.0812835574492703),
 ('##am', 0.06441275218950249),
 ('-', -0.06349643329400297),
 ('10', -0.07433490466786445),
 (':', 0.050315432791350374),
 ('00pm', 0.2952733802563707),
 ('tel', 0.18

In [ ]:
from transformers import TrainingArguments, Trainer, logging
import evaluate


# arguments for Trainer
test_args = TrainingArguments(
    output_dir = "./",
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps=256,
    eval_accumulation_steps=256,
    gradient_checkpointing=True,
    fp16=True,
    dataloader_drop_last = False    
)

# init trainer
trainer = Trainer(
              model = model, 
              eval_dataset=tokenized_datasets["test"],
              compute_metrics=compute_metrics,
              args = test_args)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BartForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2998
  Batch size = 1


{'eval_loss': 0.5371596813201904,
 'eval_accuracy': 0.7248165443629087,
 'eval_f1': 0.0,
 'eval_runtime': 106.1354,
 'eval_samples_per_second': 28.247,
 'eval_steps_per_second': 28.247}